<a href="https://colab.research.google.com/github/cincinnatilibrary/collection-analysis/blob/master/reports/percent_items_checked_out_by_location_branch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CHPL - Collection Analysis**

### Percent Checked Out by Shelf Location -- Deer Park

<img src="https://ilsweb.cincinnatilibrary.org/img/CHPL_Brandmark_Primary.png" alt="CHPL" title="CHPL" width="300"/>

Data sources

* https://collection-analysis.cincy.pl/

In [23]:
#@title
import pandas as pd
import requests
import altair as alt
import re

base_url = 'https://collection-analysis.cincy.pl'

# find the name of the target current_collection db--since it's named with its 
# hashed value
# https://datasette.io/plugins/datasette-hashed-urls
db_available = requests.get(
      url=base_url + '/-/databases' + '.json',
).json()

db_name = ''

for db in db_available:
  if re.search(r"^current_collection.*$", db['name']):
    db_name = db['name']

print(db_name)

sql = """\
with data as (
  select
    i.location_code,
    ln.name,
    -- loc.branch_code_num,
    -- bn.name as branch_name,
    count(*) as count_total_available_items,
    (
      select
        count(*)
      from
        item as i2
      where
        i2.location_code = i.location_code
        and i2.item_status_code = '-'
        and i2.checkout_total = 0
    ) as count_items_0_checkouts,
    (
      select
        count(*)
      from
        item as i2
      where
        i2.location_code = i.location_code
        and i2.item_status_code = '-'
        and i2.checkout_total > 0
    ) as count_items_gt_0_checkouts,
    (
      select
        count(*)
      from
        item as i2
      where
        i2.location_code = i.location_code
        and i2.item_status_code = '-'
        and i2.checkout_date is not null
    ) as count_curr_checked_out
  from
    item as i
    left outer join location as loc on loc.code = i.location_code
    left outer join location_name as ln on ln.location_id = loc.id
    left outer join branch as br on br.code_num = loc.branch_code_num
    left outer join branch_name as bn on bn.branch_id = br.id
  where
    i.item_status_code = '-'
    and br.code_num = :branch_code_num
  group by
    i.location_code,
    ln.name -- loc.branch_code_num,
    -- branch_name
  order by
    loc.branch_code_num
)
select
  *,
  round(
    (
      (data.count_curr_checked_out * 1.0) / (data.count_total_available_items * 1.0)
    ) * 100.0,
    2
  ) as pct_tot_curr_checked_out
from
  data
"""

df = pd.read_json(
    requests.get(
      url=base_url + '/' + db_name + '.json',
      params={
        'sql': sql,
        'branch_code_num': 11,
        '_shape': 'array'
      }
    ).content
)



current_collection


In [25]:
#@title
# alt.Chart(df[~df['branch_name'].isin(['Main Library', 'Outreach Services' ])]).mark_rect().encode(
alt.Chart(df).mark_rect().encode(
    x=alt.X(
        'name:O',
        axis=alt.Axis(labelAngle=-40),
        sort="-color"
    ),
    # y='pct_tot_curr_checked_out:O',
    color=alt.Color(
        'pct_tot_curr_checked_out:Q', 
        scale=alt.Scale(
            scheme='greens',
            domain=(0,100) 
        )
        # scale=alt.Scale(domain=(0, max_scale)),
    ),
    tooltip=['name', 'location_code', 'count_total_available_items', 'count_items_0_checkouts', 'count_items_gt_0_checkouts', 'count_curr_checked_out', 'pct_tot_curr_checked_out']
).properties(
    title="Percent Checked Out by Shelf Location -- Deer Park",
    width=900,
    # height=1000
)

alt.Chart(...)